In [5]:
import pandas as pd
import numpy as np
import datetime
import math
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from numpy import random
import operator

C:\Users\User\AppData\Local\Temp\ipykernel_7336\3045782057.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [6]:
# list of team names currently in associated leagues (including relegated teams from last season to fill DF), as they are displayed on Understat.

epl_rank = ['Manchester City', 'Arsenal', 'Liverpool', 'Manchester United', 'Newcastle United', 'Tottenham', 'Brighton',
            'Aston Villa', 'Brentford', 'Chelsea', 'Fulham', 'Crystal Palace', 'Wolverhampton Wanderers', 'Everton',
            'West Ham', 'Nottingham Forest', 'Bournemouth', 'Leicester', 'Southampton','Burnley','Luton','Sheffield United',
            'Leeds']
ligue1_rank = ['Ajaccio', 'Marseille', 'Auxerre', 'Lens', 'Monaco', 'Toulouse', 'Nantes', 'Angers', 'Lorient',
               'Strasbourg', 'Nice', 'Lyon', 'Paris Saint Germain', 'Clermont Foot', 'Brest', 'Rennes', 'Reims',
               'Montpellier', 'Troyes', 'Lille','Metz','Le Havre']
bundesliga_rank = ['Borussia Dortmund', 'Mainz 05', 'RasenBallsport Leipzig', 'Schalke 04', 'Union Berlin',
                   'Werder Bremen', 'FC Cologne', 'Bayern Munich', 'Borussia M.Gladbach', 'Augsburg',
                   'Eintracht Frankfurt', 'Freiburg', 'Wolfsburg', 'Hertha Berlin', 'Bochum', 'Bayer Leverkusen',
                   'VfB Stuttgart', 'Hoffenheim','FC Heidenheim','Darmstadt']
serieA_rank = ['Sassuolo', 'Fiorentina', 'Torino', 'Inter', 'Cremonese', 'Salernitana', 'Empoli', 'Lazio', 'Napoli',
               'Sampdoria', 'Atalanta', 'Monza', 'Lecce', 'Bologna', 'AC Milan', 'Verona', 'Roma', 'Spezia', 'Udinese',
               'Juventus','Cagliari','Frosinone','Genoa']
laliga_rank = ['Real Madrid', 'Athletic Club', 'Villarreal', 'Athletico Madrid', 'Celta Vigo', 'Barcelona',
               'Real Valladolid', 'Getafe', 'Elche', 'Cadiz', 'Espanyol', 'Almeria', 'Osasuna', 'Girona', 'Real Betis',
               'Valencia', 'Mallorca', 'Rayo Vallecano', 'Real Sociedad', 'Sevilla','Granada','Alaves','Las Palmas']

In [7]:
fixtures = pd.read_csv(r'fixtures.txt')
fixtures
TeamA = []
TeamB = []
for item in fixtures["-- Date"]:
    if "--" not in item:
        items = item.split(" v ")
        TeamA.append(items[0])
        TeamB.append(items[1])
    else:
        continue

In [251]:
# Not currently used in this script

# Machine Learning Approach (Logistic Regression) to predict winning teams
# def win_Ml(df,gamesheet_list):

#     y_win = df.win
#     x_win = df.drop(['Team','Opp','Unnamed: 0','Unnamed: 0.1','G','GC','Blank','CS','0_0','btts_if_g','btts_if_c','ID','date_int','played','date','win','draw','lose','Pts','match_ID','H/A','S','SC','QS','QSC'], axis = 1)
#     # print(x_win)
#     xtrain_win, xtest_win, ytrain_win, ytest_win = train_test_split(x_win,y_win, train_size = 0.8, random_state = 42)
#     model_win = LogisticRegression()
#     model_win.fit(xtrain_win,ytrain_win)
#     y_preds_win = model_win.predict(xtest_win)
#     accuracy_win = accuracy_score(ytest_win, y_preds_win)
#     my_params = np.array([gamesheet_list])
#     win_lk = model_win.predict(my_params)

#     return win_lk

# def draw_Ml(df,gamesheet_list):

#     y_draw = df.draw
#     x_draw = df.drop(['Team','Opp','Unnamed: 0','Unnamed: 0.1','G','GC','Blank','CS','0_0','btts_if_g','btts_if_c','ID','date_int','played','date','win','draw','lose','Pts','match_ID','H/A','S','SC','QS','QSC'], axis = 1)
#     # print(x_draw)
#     xtrain_draw, xtest_draw, ytrain_draw, ytest_draw = train_test_split(x_draw,y_draw, train_size = 0.8, random_state = 42)
#     model_draw = LogisticRegression()
#     model_draw.fit(xtrain_draw,ytrain_draw)
#     y_preds_draw = model_draw.predict(xtest_draw)
#     accuracy_draw = accuracy_score(ytest_draw, y_preds_draw)
#     my_params = np.array([gamesheet_list])
#     draw_lk = model_draw.predict(my_params)

#     return draw_lk

# def lose_Ml(df,gamesheet_list):

#     y_lose = df.lose
#     x_lose = df.drop(['Team','Opp','Unnamed: 0','Unnamed: 0.1','G','GC','Blank','CS','0_0','btts_if_g','btts_if_c','ID','date_int','played','date','win','draw','lose','Pts','match_ID','H/A','S','SC','QS','QSC'], axis = 1)
#     xtrain_lose, xtest_lose, ytrain_lose, ytest_lose = train_test_split(x_lose,y_lose, train_size = 0.8, random_state = 42)
#     model_lose = LogisticRegression()
#     model_lose.fit(xtrain_lose,ytrain_lose)
#     y_preds_lose = model_lose.predict(xtest_lose)
#     accuracy_lose = accuracy_score(ytest_lose, y_preds_lose)
#     my_params = np.array([gamesheet_list])
#     lose_lk = model_lose.predict(my_params)

#     return model_lose

In [269]:
# Not currently used as goal predictions were unrealistic

# def poisson_team_goal_prob(g_s):
#     global prob0, prob1, prob2, prob3, prob4, prob5, prob5plus
#     poisson_goals = random.poisson(lam=g_s, size=100000)
#     goals0 = 0
#     goals1 = 0
#     goals2 = 0
#     goals3 = 0
#     goals4 = 0
#     goals5 = 0
#     goals5plus = 0

#     for i in range(0, 10000):
#         if poisson_goals[i] == 0:
#             goals0 = goals0+1
#             prob0 = goals0/ 10000
#         elif poisson_goals[i] == 1:
#             goals1 = goals1+1
#             prob1 = goals1/ 10000
#         elif poisson_goals[i] == 2:
#             goals2 = goals2+1
#             prob2 = goals2/ 10000
#         elif poisson_goals[i] == 3:
#             goals3 = goals3+1
#             prob3 = goals3/ 10000
#         elif poisson_goals[i] == 4:
#             goals4 = goals4+1
#             prob4 = goals4/ 10000
#         elif poisson_goals[i] == 5:
#             goals5 = goals5+1
#             prob5 = goals5/ 10000
#         else:
#             goals5plus = goals5plus+1
#             prob5plus = goals5plus/ 10000

#     dict_g = {"0": prob0, "1": prob1, "2": prob2, "3": prob3, "4": prob4, "5": prob5, "6": prob5plus}
#     dict_max = (max(dict_g.items(), key = operator.itemgetter(1)))
#     goal_max = float(dict_max[0])

#     return goal_max

In [8]:
# Poisson Regressive model based on team blank likelihood

def poisson_reg_team_goal_prob(g_s):
    
    global prob0, prob1, prob2, prob3, prob4, prob5, prob5plus

    if g_s < 0.95:
        prob_home_1g = -(math.log(1 - g_s))
        zero_PH = 0.05  # Placeholder
        prob0 = (math.exp(-prob_home_1g)) * (pow(prob_home_1g, 0) / 1) - zero_PH
        prob1 = (math.exp(-prob_home_1g)) * (pow(prob_home_1g, 1) / 1)
        prob2 = (math.exp(-prob_home_1g)) * (pow(prob_home_1g, 2) / 2)
        prob3 = (math.exp(-prob_home_1g)) * (pow(prob_home_1g, 3) / 6) # 3!
        prob4 = (math.exp(-prob_home_1g)) * (pow(prob_home_1g, 4) / 24) # 4!
        prob5 = (math.exp(-prob_home_1g)) * (pow(prob_home_1g, 5) / 120) # 5!
        prob5plus = 1 - (prob0+prob1+prob2+prob3+prob4+prob5+zero_PH)
        dict_g1 = {"0": prob0, "1": prob1, "2": prob2, "3": prob3, "4": prob4, "5": prob5, "6": prob5plus}
        dict_max = (max(dict_g1.items(), key = operator.itemgetter(1)))
        goal_max = float(dict_max[0])
    else:
        dict_g2 = {"1": prob0, "2": prob1, "3": prob2, "4": prob3, "5": prob4, "6": prob5, "7": prob5plus}
        dict_max = (max(dict_g2.items(), key = operator.itemgetter(1)))
        goal_max = float(dict_max[0])

    return goal_max

In [9]:
def score_gen(h,a):
    # PPG
    h_ppg = h['Pts'].item()
    a_ppg = a['Pts'].item()

    # Not Blank 
    h_gnb = (1-((h['Blank'].item()+a['CS'].item())/2))
    a_gnb = (1-((a['Blank'].item()+h['CS'].item())/2))

    # Goal Difference
    h_gd = round(((h['GD'].item() - a['GD'].item())/2),0)
    a_gd = round(((a['GD'].item() - h['GD'].item())/2),0)

    # First Goal
    h_fg = (h['first_goal'].item() + a['first_conc'].item())/2
    a_fg = (a['first_goal'].item() + h['first_conc'].item())/2

    # BTTS given H/A goal
    a_btts_if_fgh = (h['btts_if_g'].item() + a['btts_if_c'].item())/2
    h_btts_if_fga = (a['btts_if_g'].item() + h['btts_if_c'].item())/2

    # If home team shows dominance in this fixture (Mismatch, away team likely to struggle)
    if (h_ppg - a_ppg) >= 1:
        if h_fg > 0.65:
            if a_btts_if_fgh < 0.5:
                # Home team expected to score first goal and control the match
                h_s = h_gd
                a_s = 0
                typescore = "Home win to nil (1)"
            else:
                # Home team expected to score first goal and likely win, but with some difficulties
                a_s = poisson_reg_team_goal_prob(a_gnb)
                h_s = (h_gd+a_s)
                typescore = "Home win (2)"
        # Unlikely, team on form that may disrupt historic results?
        elif a_fg > 0.65:
            if h_btts_if_fga < 0.5:
            # Away team expected to score first goal, home team response poor
                a_s = poisson_reg_team_goal_prob(a_gnb)
                h_s = 0
                typescore = "Away win to nil (shock result) (3)"
            else:
                # Away team score first, home team have firepower to equalise
                h_s = 1+(poisson_reg_team_goal_prob(h_gnb))
                a_s = 1+(poisson_reg_team_goal_prob(a_gnb))
                typescore = "Away team score first, Home team equalise (O2.5 BTTS) (4)"
        # No clear frontrunner
        else:
            h_s = poisson_reg_team_goal_prob(h_gnb)
            a_s = poisson_reg_team_goal_prob(a_gnb)
            typescore = "Unsure (5)"
        # If home team shows dominance in this fixture (Mismatch, away team likely to struggle)
    elif (a_ppg - h_ppg) >= 1:
        if a_fg > 0.65:
            if h_btts_if_fga < 0.5:
                # Away team expected to score first goal and control the match
                a_s = a_gd
                h_s = 0
                typescore = "Away win to nil (6)"
            else:
                # Away team expected to score first goal and likely win, but with some difficulties
                h_s = poisson_reg_team_goal_prob(h_gnb)
                a_s = (a_gd+h_s)
                typescore = "Away win (7)"
        # Unlikely, team on form that may disrupt historic results?
        elif h_fg > 0.65:
            if a_btts_if_fgh < 0.5:
            # Home team expected to score first goal, Away team response poor
                h_s = poisson_reg_team_goal_prob(h_gnb)
                a_s = 0
                typescore = "Home win to nil (shock result) (8)"
            else:
                # Home team score first, Away team have firepower to equalise
                h_s = 1+(poisson_reg_team_goal_prob(h_gnb))
                a_s = 1+(poisson_reg_team_goal_prob(a_gnb))
                typescore = "Home team score frist, Away Team equalise (BTTS) (9)"
        # No clear frontrunner
        else:
            h_s = poisson_reg_team_goal_prob(h_gnb)
            a_s = poisson_reg_team_goal_prob(a_gnb)
            typescore = "Unsure (10)"
    else:
        if a_fg > 0.65:
            if h_btts_if_fga < 0.5:
                # Away team may score first goal
                a_s = poisson_reg_team_goal_prob(a_gnb)
                h_s = 0
                typescore = "Cagey (11)"
            else:
                # Away team expected to score first goal, home team expected to equalise
                h_s = 1+(poisson_reg_team_goal_prob(h_gnb))
                a_s = 1+(poisson_reg_team_goal_prob(a_gnb))
                typescore = "BTTS (12)"
        elif h_fg > 0.65:
            if a_btts_if_fgh < 0.5:
                # Home team may score first goal
                h_s = poisson_reg_team_goal_prob(h_gnb)
                a_s = 0
                typescore = "Cagey (13)"
            else:
                # Home team expected to score first goal, Away team expected to equalise
                h_s = 1+(poisson_reg_team_goal_prob(h_gnb))
                a_s = 1+(poisson_reg_team_goal_prob(a_gnb))
                typescore = "BTTS (14)"
        # No clear frontrunner
        else:
            h_s = poisson_reg_team_goal_prob(h_gnb)
            a_s = poisson_reg_team_goal_prob(a_gnb)
            typescore = "Unsure (15)"


    return h_s, a_s, typescore

In [254]:
# Not currently used in script:


# def params_gen_ML(h,a,har,aar):
#     xGH = ((h['xG'].item() + a['xGC'].item())/2)
#     xGA = ((a['xG'].item() + h['xGC'].item())/2)
#     BTTS = ((h['BTTS'].item() + a['BTTS'].item())/2)
#     fg_h = (h['first_goal'].item() + a['first_conc'].item())/2
#     fg_a = (a['first_goal'].item() + h['first_conc'].item())/2
#     rank = har
#     rankvs = aar
#     home_list = [xGH,xGA,BTTS,fg_h,fg_a,rank,rankvs]
#     away_list = [xGA,xGH,BTTS,fg_a,fg_h,rankvs,rank]
#     return home_list, away_list



In [10]:
def which_league(HT,item):
    if HT in epl_rank:
        df = pd.read_excel(r'../EDA_DF/EPL_understat.xlsx',sheet_name='gamesheet')
        l10_table_h = pd.read_excel(r'../EDA_DF/EPL_understat.xlsx',sheet_name='last10_home')
        l10_table_a = pd.read_excel(r'../EDA_DF/EPL_understat.xlsx',sheet_name='last10_away')
        l10_df = pd.read_excel(r'../EDA_DF/EPL_understat.xlsx',sheet_name='last10_gamesheet')
        table_h = pd.read_excel(r'../EDA_DF/EPL_understat.xlsx',sheet_name='home')
        table_a = pd.read_excel(r'../EDA_DF/EPL_understat.xlsx',sheet_name='away')
        r1_table_h = pd.read_excel(r'../EDA_DF/EPL_understat.xlsx',sheet_name='rank1_home')
        r2_table_h = pd.read_excel(r'../EDA_DF/EPL_understat.xlsx',sheet_name='rank2_home')
        r3_table_h = pd.read_excel(r'../EDA_DF/EPL_understat.xlsx',sheet_name='rank3_home')
        r1_table_a = pd.read_excel(r'../EDA_DF/EPL_understat.xlsx',sheet_name='rank1_away')
        r2_table_a = pd.read_excel(r'../EDA_DF/EPL_understat.xlsx',sheet_name='rank2_away')
        r3_table_a = pd.read_excel(r'../EDA_DF/EPL_understat.xlsx',sheet_name='rank3_away')
        r1_h_df = pd.read_excel(r'../EDA_DF/EPL_understat.xlsx',sheet_name="versus_rank1_home_df")
        r2_h_df = pd.read_excel(r'../EDA_DF/EPL_understat.xlsx',sheet_name="versus_rank2_home_df")
        r3_h_df =  pd.read_excel(r'../EDA_DF/EPL_understat.xlsx',sheet_name="versus_rank3_home_df")
        r1_a_df =  pd.read_excel(r'../EDA_DF/EPL_understat.xlsx',sheet_name="versus_rank1_away_df")
        r2_a_df =  pd.read_excel(r'../EDA_DF/EPL_understat.xlsx',sheet_name="versus_rank2_away_df")
        r3_a_df =  pd.read_excel(r'../EDA_DF/EPL_understat.xlsx',sheet_name="versus_rank3_away_df")

    elif HT in ligue1_rank:
        df = pd.read_excel(r'../EDA_DF/Ligue1_understat.xlsx', sheet_name='gamesheet')
        l10_table_h = pd.read_excel(r'../EDA_DF/Ligue1_understat.xlsx',sheet_name='last10_home')
        l10_table_a = pd.read_excel(r'../EDA_DF/Ligue1_understat.xlsx',sheet_name='last10_away')
        l10_df = pd.read_excel(r'../EDA_DF/Ligue1_understat.xlsx',sheet_name='last10_gamesheet')
        table_h = pd.read_excel(r'../EDA_DF/Ligue1_understat.xlsx',sheet_name='home')
        table_a = pd.read_excel(r'../EDA_DF/Ligue1_understat.xlsx',sheet_name='away')
        r1_table_h = pd.read_excel(r'../EDA_DF/Ligue1_understat.xlsx',sheet_name='rank1_home')
        r2_table_h = pd.read_excel(r'../EDA_DF/Ligue1_understat.xlsx',sheet_name='rank2_home')
        r3_table_h = pd.read_excel(r'../EDA_DF/Ligue1_understat.xlsx',sheet_name='rank3_home')
        r1_table_a = pd.read_excel(r'../EDA_DF/Ligue1_understat.xlsx',sheet_name='rank1_away')
        r2_table_a = pd.read_excel(r'../EDA_DF/Ligue1_understat.xlsx',sheet_name='rank2_away')
        r3_table_a = pd.read_excel(r'../EDA_DF/Ligue1_understat.xlsx',sheet_name='rank3_away')
        r1_h_df = pd.read_excel(r'../EDA_DF/Ligue1_understat.xlsx', sheet_name="versus_rank1_home_df")
        r2_h_df = pd.read_excel(r'../EDA_DF/Ligue1_understat.xlsx', sheet_name="versus_rank2_home_df")
        r3_h_df =  pd.read_excel(r'../EDA_DF/Ligue1_understat.xlsx', sheet_name="versus_rank3_home_df")
        r1_a_df =  pd.read_excel(r'../EDA_DF/Ligue1_understat.xlsx', sheet_name="versus_rank1_away_df")
        r2_a_df =  pd.read_excel(r'../EDA_DF/Ligue1_understat.xlsx', sheet_name="versus_rank2_away_df")
        r3_a_df =  pd.read_excel(r'../EDA_DF/Ligue1_understat.xlsx', sheet_name="versus_rank3_away_df")

    elif HT in bundesliga_rank:
        df = pd.read_excel(r'../EDA_DF/Bundesliga_understat.xlsx', sheet_name='gamesheet')
        l10_table_h = pd.read_excel(r'../EDA_DF/Bundesliga_understat.xlsx',sheet_name='last10_home')
        l10_table_a = pd.read_excel(r'../EDA_DF/Bundesliga_understat.xlsx',sheet_name='last10_away')
        l10_df = pd.read_excel(r'../EDA_DF/Bundesliga_understat.xlsx',sheet_name='last10_gamesheet')
        table_h = pd.read_excel(r'../EDA_DF/Bundesliga_understat.xlsx',sheet_name='home')
        table_a = pd.read_excel(r'../EDA_DF/Bundesliga_understat.xlsx',sheet_name='away')
        r1_table_h = pd.read_excel(r'../EDA_DF/Bundesliga_understat.xlsx',sheet_name='rank1_home')
        r2_table_h = pd.read_excel(r'../EDA_DF/Bundesliga_understat.xlsx',sheet_name='rank2_home')
        r3_table_h = pd.read_excel(r'../EDA_DF/Bundesliga_understat.xlsx',sheet_name='rank3_home')
        r1_table_a = pd.read_excel(r'../EDA_DF/Bundesliga_understat.xlsx',sheet_name='rank1_away')
        r2_table_a = pd.read_excel(r'../EDA_DF/Bundesliga_understat.xlsx',sheet_name='rank2_away')
        r3_table_a = pd.read_excel(r'../EDA_DF/Bundesliga_understat.xlsx',sheet_name='rank3_away')
        r1_h_df = pd.read_excel(r'../EDA_DF/Bundesliga_understat.xlsx', sheet_name="versus_rank1_home_df")
        r2_h_df = pd.read_excel(r'../EDA_DF/Bundesliga_understat.xlsx', sheet_name="versus_rank2_home_df")
        r3_h_df =  pd.read_excel(r'../EDA_DF/Bundesliga_understat.xlsx', sheet_name="versus_rank3_home_df")
        r1_a_df =  pd.read_excel(r'../EDA_DF/Bundesliga_understat.xlsx', sheet_name="versus_rank1_away_df")
        r2_a_df =  pd.read_excel(r'../EDA_DF/Bundesliga_understat.xlsx', sheet_name="versus_rank2_away_df")
        r3_a_df =  pd.read_excel(r'../EDA_DF/Bundesliga_understat.xlsx', sheet_name="versus_rank3_away_df")

    elif HT in serieA_rank:
        df = pd.read_excel(r'../EDA_DF/SerieA_understat.xlsx', sheet_name='gamesheet')
        l10_table_h = pd.read_excel(r'../EDA_DF/SerieA_understat.xlsx',sheet_name='last10_home')
        l10_table_a = pd.read_excel(r'../EDA_DF/SerieA_understat.xlsx',sheet_name='last10_away')
        l10_df = pd.read_excel(r'../EDA_DF/SerieA_understat.xlsx',sheet_name='last10_gamesheet')
        table_h = pd.read_excel(r'../EDA_DF/SerieA_understat.xlsx',sheet_name='home')
        table_a = pd.read_excel(r'../EDA_DF/SerieA_understat.xlsx',sheet_name='away')
        r1_table_h = pd.read_excel(r'../EDA_DF/SerieA_understat.xlsx',sheet_name='rank1_home')
        r2_table_h = pd.read_excel(r'../EDA_DF/SerieA_understat.xlsx',sheet_name='rank2_home')
        r3_table_h = pd.read_excel(r'../EDA_DF/SerieA_understat.xlsx',sheet_name='rank3_home')
        r1_table_a = pd.read_excel(r'../EDA_DF/SerieA_understat.xlsx',sheet_name='rank1_away')
        r2_table_a = pd.read_excel(r'../EDA_DF/SerieA_understat.xlsx',sheet_name='rank2_away')
        r3_table_a = pd.read_excel(r'../EDA_DF/SerieA_understat.xlsx',sheet_name='rank3_away')
        r1_h_df = pd.read_excel(r'../EDA_DF/SerieA_understat.xlsx', sheet_name="versus_rank1_home_df")
        r2_h_df = pd.read_excel(r'../EDA_DF/SerieA_understat.xlsx', sheet_name="versus_rank2_home_df")
        r3_h_df =  pd.read_excel(r'../EDA_DF/SerieA_understat.xlsx', sheet_name="versus_rank3_home_df")
        r1_a_df =  pd.read_excel(r'../EDA_DF/SerieA_understat.xlsx', sheet_name="versus_rank1_away_df")
        r2_a_df =  pd.read_excel(r'../EDA_DF/SerieA_understat.xlsx', sheet_name="versus_rank2_away_df")
        r3_a_df =  pd.read_excel(r'../EDA_DF/SerieA_understat.xlsx', sheet_name="versus_rank3_away_df")

    elif HT in laliga_rank:
        df = pd.read_excel(r'../EDA_DF/LaLiga_understat.xlsx', sheet_name='gamesheet')
        l10_table_h = pd.read_excel(r'../EDA_DF/LaLiga_understat.xlsx',sheet_name='last10_home')
        l10_table_a = pd.read_excel(r'../EDA_DF/LaLiga_understat.xlsx',sheet_name='last10_away')
        l10_df = pd.read_excel(r'../EDA_DF/LaLiga_understat.xlsx',sheet_name='last10_gamesheet')
        table_h = pd.read_excel(r'../EDA_DF/LaLiga_understat.xlsx',sheet_name='home')
        table_a = pd.read_excel(r'../EDA_DF/LaLiga_understat.xlsx',sheet_name='away')
        r1_table_h = pd.read_excel(r'../EDA_DF/LaLiga_understat.xlsx',sheet_name='rank1_home')
        r2_table_h = pd.read_excel(r'../EDA_DF/LaLiga_understat.xlsx',sheet_name='rank2_home')
        r3_table_h = pd.read_excel(r'../EDA_DF/LaLiga_understat.xlsx',sheet_name='rank3_home')
        r1_table_a = pd.read_excel(r'../EDA_DF/LaLiga_understat.xlsx',sheet_name='rank1_away')
        r2_table_a = pd.read_excel(r'../EDA_DF/LaLiga_understat.xlsx',sheet_name='rank2_away')
        r3_table_a = pd.read_excel(r'../EDA_DF/LaLiga_understat.xlsx',sheet_name='rank3_away')
        r1_h_df = pd.read_excel(r'../EDA_DF/LaLiga_understat.xlsx', sheet_name="versus_rank1_home_df")
        r2_h_df = pd.read_excel(r'../EDA_DF/LaLiga_understat.xlsx', sheet_name="versus_rank2_home_df")
        r3_h_df =  pd.read_excel(r'../EDA_DF/LaLiga_understat.xlsx', sheet_name="versus_rank3_home_df")
        r1_a_df =  pd.read_excel(r'../EDA_DF/LaLiga_understat.xlsx', sheet_name="versus_rank1_away_df")
        r2_a_df =  pd.read_excel(r'../EDA_DF/LaLiga_understat.xlsx', sheet_name="versus_rank2_away_df")
        r3_a_df =  pd.read_excel(r'../EDA_DF/LaLiga_understat.xlsx', sheet_name="versus_rank3_away_df")
        
    else:
        print("This Team doesnt feature in our database!")
        item += 1
    
    return df, l10_table_h, l10_table_a, l10_df, table_h, table_a, r1_table_h, r2_table_h, r3_table_h,r1_table_a, r2_table_a, r3_table_a, r1_h_df, r2_h_df, r3_h_df, r1_a_df, r2_a_df, r3_a_df

In [11]:
def decision_tree(Home_Team, Away_Team,hf_rank,hrec,hf_rankvs,r1_table_a,r1_a_df,r2_table_a,r2_a_df,r3_table_a,r3_a_df,af_rank,arec,af_rankvs,r1_table_h,r1_h_df,r2_table_h,r2_h_df,r3_table_h,r3_h_df):
    if (hf_rank == hrec) or (hf_rank < hrec and hf_rankvs > 2.2) or (hf_rank > hrec and hf_rankvs < 1.8):
        if hf_rank == 1.0:
            home_adj_rank = 1.0
            away = r1_table_a.loc[r1_table_a["Team"] == Away_Team]
            away_df = r1_a_df.loc[r1_a_df["Team"] == Away_Team]
        elif hf_rank == 2.0:
            home_adj_rank = 2.0
            away = r2_table_a.loc[r2_table_a["Team"] == Away_Team]
            away_df = r2_a_df.loc[r2_a_df["Team"] == Away_Team]
        else:
            home_adj_rank = 3.0
            away = r3_table_a.loc[r3_table_a["Team"] == Away_Team]
            away_df = r3_a_df.loc[r3_a_df["Team"] == Away_Team]
    elif ((hf_rank < hrec) and (hf_rankvs <= 2.2)):
        if hf_rank == 1.0:
            home_adj_rank = 1.0
            away = r1_table_a.loc[r1_table_a["Team"] == Away_Team]
            away_df = r1_a_df.loc[r1_a_df["Team"] == Away_Team]
        else:
            home_adj_rank = 2.0
            away = r2_table_a.loc[r2_table_a["Team"] == Away_Team]
            away_df = r2_a_df.loc[r2_a_df["Team"] == Away_Team]
    elif ((hf_rank > hrec) and (hf_rankvs >= 1.8)):
        if hf_rank == 2.0:
            home_adj_rank = 2.0
            away = r2_table_a.loc[r2_table_a["Team"] == Away_Team]
            away_df = r2_a_df.loc[r2_a_df["Team"] == Away_Team]
        else:
            home_adj_rank = 3.0
            away = r3_table_a.loc[r3_table_a["Team"] == Away_Team]
            away_df = r3_a_df.loc[r3_a_df["Team"] == Away_Team]
    else:
        print("???")

        # Away
            
    if (af_rank == arec) or (af_rank < arec and af_rankvs > 2.25) or (af_rank > arec and af_rankvs < 1.75):
        if af_rank == 1.0:
            away_adj_rank = 1.0
            home = r1_table_h.loc[r1_table_h["Team"] == Home_Team]
            home_df = r1_h_df.loc[r1_h_df["Team"] == Home_Team]
        elif af_rank == 2.0:
            away_adj_rank = 2.0
            home = r2_table_h.loc[r2_table_h["Team"] == Home_Team]
            home_df = r2_h_df.loc[r2_h_df["Team"] == Home_Team]
        else:
            away_adj_rank = 3.0
            home = r3_table_h.loc[r3_table_h["Team"] == Home_Team]
            home_df = r3_h_df.loc[r3_h_df["Team"] == Home_Team]
    elif ((af_rank < arec) and (af_rankvs <= 2.25)):
        if af_rank == 1.0:
            away_adj_rank = 1.0
            home = r1_table_h.loc[r1_table_h["Team"] == Home_Team]
            home_df = r1_h_df.loc[r1_h_df["Team"] == Home_Team]
        else:
            away_adj_rank = 2.0
            home = r2_table_h.loc[r2_table_h["Team"] == Home_Team]
            home_df = r2_h_df.loc[r2_h_df["Team"] == Home_Team]
    elif ((af_rank > arec) and (af_rankvs >= 1.75)):
        if af_rank == 2.0:
            away_adj_rank = 2.0
            home = r2_table_h.loc[r2_table_h["Team"] == Home_Team]
            home_df = r2_h_df.loc[r2_h_df["Team"] == Home_Team]
        else:
            away_adj_rank = 3.0
            home = r3_table_h.loc[r3_table_h["Team"] == Home_Team]
            home_df = r3_h_df.loc[r3_h_df["Team"] == Home_Team]
    else:
        print("???")
    return home, away, home_adj_rank, away_adj_rank, home_df, away_df

In [12]:

def rank_decision():
    
    scoresheet_df = pd.DataFrame()
    teamlist = []
    h_score = []
    a_score = []
    pred_type = []
    h_rank = []
    a_rank = []

    for i in range(0,len(TeamA)):
        Home_Team = TeamA[i]
        Away_Team = TeamB[i]

        df, l10_table_h, l10_table_a, l10_df, table_h, table_a, r1_table_h, r2_table_h, r3_table_h,r1_table_a, r2_table_a, r3_table_a, r1_h_df, r2_h_df, r3_h_df, r1_a_df, r2_a_df, r3_a_df = which_league(Home_Team,i)
        
        # Historic Rank data vs Form-Influenced Rank Data
            
        hf = l10_table_h.loc[l10_table_h["Team"] == Home_Team]
        hrec = table_h.loc[table_h["Team"] == Home_Team]
        af = l10_table_a.loc[l10_table_a["Team"] == Away_Team]
        arec = table_a.loc[table_a["Team"] == Away_Team]
        hf_rank = float(hf['Rank'].item()) 
        hf_rankvs = float(hf['Rankvs'].item()) 
        hrec = float(hrec['Rank'].item()) 
        af_rank = float(af['Rank'].item()) 
        af_rankvs = float(af['Rankvs'].item()) 
        arec = float(arec['Rank'].item()) 

        # Home

        # If Form and Historic ranks are the same (or if form suggests a different rank, but the fixture diffiulty is abnormal)
        home, away, home_adj_rank, away_adj_rank, home_df, away_df = decision_tree(Home_Team, Away_Team,hf_rank,hrec,hf_rankvs,r1_table_a,r1_a_df,r2_table_a,r2_a_df,r3_table_a,r3_a_df,af_rank,arec,af_rankvs,r1_table_h,r1_h_df,r2_table_h,r2_h_df,r3_table_h,r3_h_df)
       
        score_h,score_a,type_score = score_gen(home,away)
        
        teamlist.append(f'{Home_Team} v {Away_Team}')
        h_score.append(score_h)
        a_score.append(score_a)
        pred_type.append(type_score)
        h_rank.append(home_adj_rank)
        a_rank.append(away_adj_rank)

    scoresheet_df['Team'] = teamlist
    scoresheet_df['Home Score'] = h_score
    scoresheet_df['Away Score'] = a_score
    scoresheet_df['Type Score'] = pred_type
    scoresheet_df['Home Rank'] = h_rank
    scoresheet_df['Away Rank'] = a_rank

    return scoresheet_df


scoresheet = rank_decision()
scoresheet

,Team,Home Score,Away Score,Type Score,Home Rank,Away Rank
0,Newcastle United v West Ham,2.0,1.0,Unsure (15),3.0,2.0
1,Bournemouth v Everton,2.0,3.0,BTTS (12),3.0,3.0
2,Chelsea v Burnley,2.0,1.0,Home win (2),1.0,3.0
3,Nottingham Forest v Crystal Palace,2.0,1.0,Home win (2),2.0,3.0
4,Sheffield United v Fulham,1.0,2.0,Away win (7),3.0,2.0
5,Tottenham v Luton,2.0,1.0,Home win (2),1.0,2.0
6,Aston Villa v Wolverhampton Wanderers,2.0,0.0,Cagey (13),2.0,1.0
7,Brentford v Manchester United,1.0,2.0,Away win (7),3.0,2.0
8,Liverpool v Brighton,2.0,1.0,Home win (2),1.0,2.0
9,Manchester City v Arsenal,2.0,2.0,Unsure (5),1.0,1.0


In [330]:
date = datetime.datetime.now().strftime("%m-%d-%Y")

with pd.ExcelWriter("../Frontend/scorepredictions.xlsx", mode="a") as writer:
    scoresheet.to_excel(writer, sheet_name=f"gamesheet_{date}")